# Test the synthetic user's trajectories
- trajectories are generated by using starting and ending points the **medoids** computed with  notebook as they represent real-world aggregation points
- trajectories are interpolated and calculate timedeltas between each point in order to fake a realistic trajectory according to various mobility profiles.

Below:
1. Example of trajectory generation

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import yaml
import osmnx as ox

with open("conf.yaml") as f:
    conf = yaml.load(f, Loader = yaml.FullLoader)
    
out_path = conf["out_path"]
data_path = conf["data_path"]
    


In [ ]:
# lead the original medoids
medoids = pd.read_csv(out_path + "geolife_medoids.csv")
print(medoids.head())

## 1. Example of Trajectory generation
    1.1. Set **S**tarting and **A**rrival point with 2 medoids
    1.2 Plot on the map
    1.3 Define the shortest path between **S** and **A**
    1.4 interpolate gps points between **S** and **A**

In [ ]:
med_0 = (medoids.iloc[0].lat, medoids.iloc[0].lng)
med_8 = (medoids.iloc[40].lat, medoids.iloc[40].lng)
print("Starting medoid: " + str(med_0))
print("Arrival medoid: " + str(med_8))

### 1.2 We plot the two medoids on Beijing's map:

In [ ]:
#obtaining Beijing's drive map
# Very slow, just uncomment once
G = ox.graph_from_place('Beijing', network_type='walk',simplify=True)

fig, ax = ox.plot_graph(G, 
                        show=False, close=False, 
                        edge_color="green")
ax.scatter(med_0[1], med_0[0], c="red", s=100)
ax.scatter(med_8[1], med_8[0], c="blue", s=100)
#plt.show()

### 1.3 Given the graph G, we need to find the closest existing points of G with respect to the **S**ource and the **A**rrival medoids. This points allow to calculate the shortest path between them

In [ ]:
from sklearn.neighbors import KDTree
# convert the multi graph to GeoDataFrames
gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)

tree = KDTree(gdf_nodes[['y', 'x']], metric='euclidean')

med_0_idx = tree.query([med_0], k=1, return_distance=False)[0]
med_8_idx = tree.query([med_8], k=1, return_distance=False)[0]

closest_node_to_0 = gdf_nodes.iloc[med_0_idx].index.values[0]
closest_node_to_8 = gdf_nodes.iloc[med_8_idx].index.values[0]

Now we try to calculate the shortest path between these two points

In [ ]:
import networkx as nx

path = nx.shortest_path(G, 
                         closest_node_to_0,
                         closest_node_to_8,
                         weight='length')
#gdf_nodes.loc[path]

print (nx.shortest_path_length(G, 
                         closest_node_to_0,
                         closest_node_to_8,
                         weight='length'))

print(path)

Let's print the path on the graph and then on a folium map:

In [ ]:
fig, ax = ox.plot_graph_route(G, path, 
                              show=False, close=False, 
                              edge_color='green',
                              route_color="purple")
plt.show()

In [ ]:
import folium

m = ox.plot_route_folium(G, path, route_color='green')
folium.Marker(location=med_0,
              icon=folium.Icon(color="red")).add_to(m)
folium.Marker(location=med_8,
              icon=folium.Icon(color='blue')).add_to(m)
m

### 1.4 interpolation to get dense trajectories.
First we show our path and relative geodataframe.

In [ ]:
gdf = gdf_nodes.loc[path]

print(gdf.head())
print(len(gdf))

We're ready to try interpolation on our points, in order to to so we employ geographiclib.
We try and get a point every k meters (i.e. 100m).

If we put 10m everything would just explode because of folium, this is just for demo purposes.

In [ ]:
from geographiclib.geodesic import Geodesic
import math

m = folium.Map(tiles="Stamen Toner", zoom_start=2)

k = 500 #interpolation offset, over k meters generate lon lat

geod = Geodesic.WGS84

for i in range(len(gdf) - 1):
    
    l = geod.InverseLine(gdf.iloc[i].y, gdf.iloc[i].x, gdf.iloc[i+1].y, gdf.iloc[i+1].x)
    ds = k; n = int(math.ceil(l.s13 / ds))
    for i in range(n + 1):
        #if i == 0:
            #print( "distance latitude longitude azimuth")
        s = min(ds * i, l.s13)
        g = l.Position(s, Geodesic.STANDARD | Geodesic.LONG_UNROLL)
        folium.Marker([g["lat2"], g["lon2"]]).add_to(m)
        #print ("{:.0f} {:.5f} {:.5f} {:.5f}".format(
        #    g['s12'], g['lat2'], g['lon2'], g['azi2']))


m